In [ ]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup  
import re
import sqlalchemy as db
from pathlib import Path

<h4>Connect to server</h4>

In [ ]:
username = "root"
password = "password"
server = "localhost"
database = "airplane_crashes"

engine = db.create_engine(f"mysql+pymysql://{username}:{password}@{server}/{database}")
metadata = db.MetaData()

crashes_tbl = db.Table('Crashes', metadata, autoload_with=engine)

<h4>Scout data</h4>

In [ ]:
website = "http://www.planecrashinfo.com/"
source = Path('database.htm')
soup = BeautifulSoup(requests.get(website + str(source)).content)

In [ ]:
print(soup.prettify())

In [ ]:
tables = soup.find_all('table')
tables

<h4>Data description</h4>

In [ ]:
descr = []
for line in tables[0].find_all('li'):
    descr.append(line.text.strip())
descr

<h4>Column description</h4>

In [ ]:
data_format = pd.read_html(str(tables))[-1]
data_format.columns = ['field', 'description']
data_format = data_format.iloc[1:, :]
data_format.field = data_format.field.apply(lambda x: x[:-1])
data_format

<h4>Process Data</h4>

In [ ]:
def process_record(index, con):
    
    df = pd.read_html(requests.get(website + str(index)).content)[0]
    value = df.iloc[1:, :].set_index(0).iloc[:, 0]
    
    value.index = list(map(lambda x: x[:-1].lower(), value.index))
    value = value.rename({'flight #': 'flight_no', 'ac  type': 'ac_type', 'cn / ln': 'cn/ln'})

    stmt = db.insert(crashes_tbl).values(**{key:val for key, val in value.to_dict().items() if val != '?'})
    con.execute(stmt)

In [ ]:
def process_year(index, con):

    parent = index.parent
    soup = BeautifulSoup(requests.get(website + str(index)).content)
    
    for row in soup.find_all('tr'):
        
        if row.a is not None:
            process_record(parent / Path(row.a['href']), con)

In [ ]:
date_regex = re.compile(r'\d{4}')

with engine.connect() as con:
    for row in tables[1].find_all('td'):
        date = row.text.strip()
        if date_regex.match(date):
            rel_link = Path(row.a['href'])
            process_year(rel_link, con)